# Ollama Environment Setup

This cell sets up the Python environment and connects to your local Ollama server.
It installs required packages, sets model and host variables, and prepares the Ollama client for use.

**Purpose:**
- Ensures all dependencies are installed and variables are set.
- Initializes the Ollama client so you can send prompts to the model.
- Used as a setup step in other tutorial notebooks via `%run 00_Tutorial_How-To.ipynb`.
- Run this cell first before using any prompt engineering or model interaction cells.

Refer to the project [README](../readme.md) for instructions on installing the local Ollama server.

---

## Usage Notes & Tips 💡
- This tutorial uses Qwen 2.5 7B Instruct with temperature 0. You can change the model name to any available in the [Ollama model library](https://ollama.com/search).
- Use `Shift + Enter` to execute the cell and move to the next one.

### The Ollama Python Library
We will be using the [Ollama Python Library](https://github.com/ollama/ollama-python) throughout this tutorial.

In [ ]:
if 'client' not in globals():
    # Install required dependencies
    %pip install -U ollama tqdm pickleshare nbformat --quiet
    %pip install python-dotenv --quiet

    # Load environment variables from .env file
    from dotenv import load_dotenv
    import os
    # Connect to Ollama Service
    from ollama import Client, ChatResponse, Options
    from IPython.display import display, Markdown

    load_dotenv()  # This loads variables from .env into the environment
    output = []

    # Now you can use OLLAMA_HOST from the .env file
    OLLAMA_HOST = os.getenv('OLLAMA_HOST')
    client = Client(host=OLLAMA_HOST)
    output.append(f'**Connected to Ollama at:** {OLLAMA_HOST}')
    # Set up your model name from .env or use default
    MODEL_NAME = os.getenv('MODEL_NAME', 'qwen2.5:14b-instruct')
    output.append(f'**Using model:** {MODEL_NAME}')
    display(Markdown("\n".join(output)))
else:
    from IPython.display import display, Markdown
    output = []
    output.append(f'**Using Ollama host:** {client._client.base_url}')
    output.append("**Ollama client already initialized.**")
    display(Markdown("\n".join(output)))

# Helper function to send a prompt or conversation to Ollama and get the response
def get_completion(prompt_or_messages, system_prompt="", max_tokens=200, temperature=0.0):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    if isinstance(prompt_or_messages, str):
        messages.append({"role": "user", "content": prompt_or_messages})
    elif isinstance(prompt_or_messages, list):
        messages = prompt_or_messages
    else:
        raise ValueError("prompt_or_messages must be a string or a list of dicts")

    # 🔎 DEBUG: display what we send
    from IPython.display import display, Markdown
    debug_str = f"\n=== Sending to Ollama ===\n{messages}\n=========================\n"
    display(Markdown(f"```\n{debug_str}\n```"))

    response: ChatResponse = client.chat(
        model=MODEL_NAME,
        options=Options(max_tokens=max_tokens, temperature=temperature),
        messages=messages
    )
    return response

def pretty_print_response(response: ChatResponse):
    from datetime import datetime  # For friendly timestamp formatting
    def ns_to_sec(ns):  # Convert nanoseconds to seconds for readability
        return f"{ns/1e9:.2f} s" if ns is not None else None
    output = []
    output.append(f"**Model:** {response.model}")
    # Timestamp
    if response.created_at:
        try:
            dt = datetime.fromisoformat(response.created_at.replace('Z', '+00:00'))
            output.append(f"**Created at:** {dt.strftime('%Y-%m-%d %H:%M:%S UTC')}")
        except Exception:
            output.append(f"**Created at:** {response.created_at}")
    else:
        output.append(f"**Created at:** {response.created_at}")
    output.append(f"**Done:** {response.done}")
    output.append(f"**Done reason:** {response.done_reason}")
    output.append(f"**Total duration:** {ns_to_sec(response.total_duration)}")
    output.append(f"**Load duration:** {ns_to_sec(response.load_duration)}")
    output.append(f"**Prompt eval count:** {response.prompt_eval_count}")
    output.append(f"**Prompt eval duration:** {ns_to_sec(response.prompt_eval_duration)}")
    output.append(f"**Eval count:** {response.eval_count}")
    output.append(f"**Eval duration:** {ns_to_sec(response.eval_duration)}")
    msg = response.message
    if msg:
        output.append(f"**Message Role:** {msg.role}")
        output.append(f"**Message Content:**\n\n{msg.content}")
    else:
        output.append("**Full Response Object:**\n\n" + str(response))
    from IPython.display import display, Markdown
    display(Markdown("\n".join(output)))
